In [21]:
from datetime import timedelta, datetime
import os
import pandas as pd
import numpy as np

In [25]:
data_path = "../data/1min/"
stocks = [file.split('/')[-1][:-4] for file in os.listdir(data_path)]
datetime_format = "%Y-%m-%dT%X+03:00"

In [7]:
# read csv of SISE
sisecam_data = pd.read_csv(data_path + "SISE.csv")

In [28]:
sisecam_data.tail()

,Unnamed: 0,symbol,signal_time,date_time,close,high,low,open,total_quantity,volume,weighted_average
396099,6099,SISE,1609167360000,2020-12-28T17:56:00+03:00,7.24,7.25,7.23,7.24,1186305.0,8.588728e+06,7.239899
396100,6100,SISE,1609167420000,2020-12-28T17:57:00+03:00,7.25,7.25,7.24,7.24,85510.0,6.195058e+05,7.244835
396101,6101,SISE,1609167480000,2020-12-28T17:58:00+03:00,7.25,7.25,7.24,7.24,150157.0,1.088206e+06,7.247118
396102,6102,SISE,1609167540000,2020-12-28T17:59:00+03:00,7.25,7.25,7.24,7.24,455938.0,3.302788e+06,7.243942
396103,6103,SISE,1609167840000,2020-12-28T18:04:00+03:00,7.25,7.25,7.25,7.25,4958976.0,3.595258e+07,7.250000


In [9]:
print(sisecam_data.shape)
print(sisecam_data.size / 10**6, " MBs")

(396104, 11)
4.357144  MBs


In [14]:
import datetime
date = datetime.datetime.strptime("2020-12-28T17:56:00+03:00", "%Y-%m-%dT%X+03:00")


In [19]:
(date + datetime.timedelta(minutes=1)).strftime("%Y-%m-%dT%X+03:00")

'2020-12-28T17:57:00+03:00'

In [26]:
for date in sisecam_data["date_time"][:100]:
    dt = datetime.strptime(date, datetime_format)
    print(dt)


2017-01-02 10:00:00
2017-01-02 10:03:00
2017-01-02 10:04:00
2017-01-02 10:05:00
2017-01-02 10:06:00
2017-01-02 10:07:00
2017-01-02 10:08:00
2017-01-02 10:09:00
2017-01-02 10:10:00
2017-01-02 10:12:00
2017-01-02 10:13:00
2017-01-02 10:14:00
2017-01-02 10:16:00
2017-01-02 10:19:00
2017-01-02 10:20:00
2017-01-02 10:21:00
2017-01-02 10:22:00
2017-01-02 10:27:00
2017-01-02 10:30:00
2017-01-02 10:31:00
2017-01-02 10:32:00
2017-01-02 10:34:00
2017-01-02 10:35:00
2017-01-02 10:36:00
2017-01-02 10:37:00
2017-01-02 10:38:00
2017-01-02 10:39:00
2017-01-02 10:40:00
2017-01-02 10:41:00
2017-01-02 10:43:00
2017-01-02 10:44:00
2017-01-02 10:45:00
2017-01-02 10:46:00
2017-01-02 10:47:00
2017-01-02 10:48:00
2017-01-02 10:50:00
2017-01-02 10:52:00
2017-01-02 10:53:00
2017-01-02 10:54:00
2017-01-02 10:57:00
2017-01-02 11:00:00
2017-01-02 11:01:00
2017-01-02 11:02:00
2017-01-02 11:03:00
2017-01-02 11:04:00
2017-01-02 11:05:00
2017-01-02 11:06:00
2017-01-02 11:07:00
2017-01-02 11:10:00
2017-01-02 11:11:00


In [65]:
a = pd.DataFrame([], columns=sisecam_data.columns.values)
for index, row in sisecam_data[1:].iterrows():
    print(row)
    a = a.append(row)
    break
a[['symbol', 'signal_time']]

Unnamed: 0                                  1
symbol                                   SISE
signal_time                     1483340580000
date_time           2017-01-02T10:03:00+03:00
close                                3.085876
high                                 3.085876
low                                  3.085876
open                                 3.085876
total_quantity                   21121.017034
volume                           65176.838861
weighted_average                     3.085876
Name: 1, dtype: object


,symbol,signal_time
1,SISE,1483340580000


In [160]:
class Imputer:
    
    from datetime import datetime
    
    START_DATE = datetime.strptime("2017-01-02T10:00:00+03:00", "%Y-%m-%dT%X+03:00")
    START_TIMESTAMP = 1483340400000
    END_DATE = datetime.strptime("2020-12-28T18:00:00+03:00", "%Y-%m-%dT%X+03:00")
    END_TIMESTAMP = 1609167600000
    
    def __init__(self, df):
        self.START_DATE = datetime.strptime("2017-01-02T10:00:00+03:00", "%Y-%m-%dT%X+03:00")
        self.START_TIMESTAMP = 1483340400000
        self.END_DATE = datetime.strptime("2020-12-28T18:00:00+03:00", "%Y-%m-%dT%X+03:00")
        self.END_TIMESTAMP = 1609167600000
    
        self.data_with_missing_values = df
        self.total_missings = 0
        self.total_samples = df.shape[0]
        self.symbol = df["symbol"][0]
        self.saved_columns = [
            'signal_time',
            'date_time',
            'close',
            'high',
            'low',
            'open',
            'total_quantity',
            'weighted_average'
        ]
        self.imputed_dataset = pd.DataFrame([], columns=self.saved_columns)
        self.reported_missing_values = []
    
    def interpolate(self, start_value, end_value, interval):
        
        missing_values = []
        power = 1/(interval+1)
        
        for i in range(interval):
            missing_values.append((start_value**interval-i + end_value**(i+1)) ** power)
        
        self.total_missings += interval
        
        return missing_values
    
    def insert(self, row):
        self.imputed_dataset = self.imputed_dataset.append(row[self.saved_columns])
    
    def fill_missing_values(self):
        
        start = self.START_TIMESTAMP
        end = self.END_TIMESTAMP
        
        prev_row = self.data_with_missing_values.iloc[0][self.saved_columns]

        for _, new_row in self.data_with_missing_values[1:].iterrows():
            
            stamp = new_row['signal_time']
            
            if (stamp - start) == 60000:
                self.insert(prev_row)
                start, prev_row = stamp, new_row
                continue
            
            elif (stamp - start) / 60000 > 10:
                # more than 10 minutes missing
                start_date = datetime.fromtimestamp(start // 1000)
                stamp_date = datetime.fromtimestamp(stamp // 1000)
                
                if (stamp_date.hour == 10 and stamp_date.minute < 10):
                    # day changed
                    self.insert(prev_row)
                    start, prev_row = stamp, new_row
                elif (
                    (stamp_date.hour == 14 and stamp_date.minute < 5) or 
                    (stamp_date.hour == 13 and stamp_date.minute > 55)
                ):
                    # session changed
                    self.insert(prev_row)
                    start, prev_row = stamp, new_row
                else:
                    # report value if day not changed
                    self.reported_missing_values.append((stamp, prev_row))
                    start, prev_row = stamp, new_row
            else:
                debug_checker = 0
                while start != stamp:
                    debug_checker += 1
                    if debug_checker > 10:
                        self.reported_missing_values.append(('UNEXPECTED', stamp, new_row))
                        break
                    prev_row['signal_time'] = start
                    self.insert(prev_row)
                    
                    start += 60000
                start, prev_row = stamp, new_row


In [161]:
imputer = Imputer(sisecam_data)

In [162]:
imputer.total_samples

396104

In [163]:
imputer.fill_missing_values()

KeyboardInterrupt: 

In [126]:
imputer.reported_missing_values

[]

In [128]:
imputer.imputed_dataset

,signal_time,date_time,close,high,low,open,total_quantity,weighted_average
0,1483340400000,2017-01-02T10:00:00+03:00,3.085876,3.093954,3.077798,3.077798,50377.500247,3.086321
0,1483340460000,2017-01-02T10:00:00+03:00,3.085876,3.093954,3.077798,3.077798,50377.500247,3.086321
0,1483340520000,2017-01-02T10:00:00+03:00,3.085876,3.093954,3.077798,3.077798,50377.500247,3.086321


In [143]:
sisecam_data.head(15)

,Unnamed: 0,symbol,signal_time,date_time,close,high,low,open,total_quantity,volume,weighted_average
0,0,SISE,1483340400000,2017-01-02T10:00:00+03:00,3.085876,3.093954,3.077798,3.077798,50377.500247,155481.137262,3.086321
1,1,SISE,1483340580000,2017-01-02T10:03:00+03:00,3.085876,3.085876,3.085876,3.085876,21121.017034,65176.838861,3.085876
2,2,SISE,1483340640000,2017-01-02T10:04:00+03:00,3.077798,3.085876,3.077798,3.085876,9747.209479,30059.939492,3.083953
3,3,SISE,1483340700000,2017-01-02T10:05:00+03:00,3.085876,3.093954,3.085876,3.085876,136168.788755,420499.992371,3.088079
4,4,SISE,1483340760000,2017-01-02T10:06:00+03:00,3.077798,3.077798,3.077798,3.077798,1237.898080,3809.999943,3.077798
5,5,SISE,1483340820000,2017-01-02T10:07:00+03:00,3.077798,3.085876,3.077798,3.085876,17959.425339,55300.479146,3.079190
6,6,SISE,1483340880000,2017-01-02T10:08:00+03:00,3.077798,3.077798,3.077798,3.077798,9053.986554,27866.339581,3.077798
7,7,SISE,1483340940000,2017-01-02T10:09:00+03:00,3.061641,3.077798,3.061641,3.077798,21949.170849,67379.949152,3.069818
8,8,SISE,1483341000000,2017-01-02T10:10:00+03:00,3.069720,3.069720,3.061641,3.069720,43388.327690,133189.498329,3.069708
9,9,SISE,1483341120000,2017-01-02T10:12:00+03:00,3.061641,3.061641,3.061641,3.061641,12256.428886,37524.789622,3.061641


In [130]:
a, b = 1, 2

In [131]:
a

1

In [132]:
b

2